In [ ]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_batch_boi")' SchwiftyNotebook_batch_boi

Installing packages:
	.package(path: "/home/ubuntu/workspace/fast-ai-swift/SchwiftyNotebook_batch_boi")
		SchwiftyNotebook_batch_boi
With SwiftPM flags: []
Working in: /tmp/tmp4bhf14xo/swift-install
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information availabl

In [ ]:
//export
import Path
import TensorFlow
import SchwiftyNotebook_batch_boi

In [ ]:
public typealias SingleValueAndGradientChain<T>=(value: T, gradientChain: (T) -> T )
public typealias SingleInputDifferentiable<T> = (T) -> SingleValueAndGradientChain<T>

In [ ]:
let identititySVGC: SingleInputDifferentiable<TensorFloat> = { x in (value: x, gradientChain: { y in y }) }

In [ ]:
struct HyperParameter {
    let learningRate: Float
    
    init(learningRate: Float){
        self.learningRate = learningRate
    }
}

In [8]:
struct RuntimeError: Error {
    let message: String

    init(_ message: String) {
        self.message = message
    }

    public var localizedDescription: String {
        return message
    }
}

In [13]:
typealias ThreeInputGradient = (TensorFloat) -> (TensorFloat, TensorFloat, TensorFloat)

struct DenseLayer {
    let activationFunction: SingleInputDifferentiable<TensorFloat>
    let weightParameter: TensorFloat
    let biasParameter: TensorFloat
    private let gradientChain: ThreeInputGradient

    private init(activationFunction: @escaping SingleInputDifferentiable<TensorFloat>,
                 weightParameter: TensorFloat,
                 biasParameter: TensorFloat,
                 gradientChain: @escaping ThreeInputGradient) {
        self.activationFunction = activationFunction
        self.weightParameter = weightParameter
        self.biasParameter = biasParameter
        self.gradientChain = gradientChain
    }

    private init(denseLayer: DenseLayer,
                 gradientChain: @escaping ThreeInputGradient) {
        self.activationFunction = denseLayer.activationFunction
        self.weightParameter = denseLayer.weightParameter
        self.biasParameter = denseLayer.biasParameter
        self.gradientChain = gradientChain
    }

    init(inputSize: Int,
         outputSize: Int,
         activationFunction: @escaping SingleInputDifferentiable<TensorFloat> = identititySVGC){
        self.activationFunction = activationFunction
        self.weightParameter = TensorFloat(kaimingUniform: TensorShape([inputSize, outputSize]))
        self.biasParameter = TensorFloat(zeros: [outputSize])
        self.gradientChain = { (x: TensorFloat)  in (TensorFloat([1]), TensorFloat([1]), TensorFloat([1]))}
    }

    func forwardPass(inputTensor: TensorFloat) -> (TensorFloat, DenseLayer) {
        let (parameterOutput, parameterGradientChain) =
          linearCombinationAndGradient(inputTensor: inputTensor,
                                       weightParameter: self.weightParameter,
                                       biasParameter: self.biasParameter)
        let (activations, activationsGradientChain) = activationFunction(parameterOutput)

        return (activations, DenseLayer(denseLayer: self,
                                       gradientChain: {ddx in parameterGradientChain(activationsGradientChain(ddx))}))
    }

    public func callAsFunction(_ inputTensor: TensorFloat) -> TensorFloat {
        let parameterOutput =
          linearCombination(inputs: inputTensor,
                                       weights: self.weightParameter,
                                       bias: self.biasParameter)
        let (activations, activationsGradientChain) = activationFunction(parameterOutput)
        return activations
    }

    func backwardPass(ddx: TensorFloat, hyperParams: HyperParameter) -> (TensorFloat, DenseLayer) {
        let (ddxInput, ddxParameter, ddxBias) = self.gradientChain(ddx)
        return (ddxInput, DenseLayer(activationFunction: activationFunction,
                                    weightParameter: self.weightParameter - hyperParams.learningRate * ddxParameter,
                                    biasParameter: self.biasParameter - hyperParams.learningRate * ddxBias,
                                    gradientChain: self.gradientChain))
    }
}

In [14]:
let usedCarDataBunch = fetchUsedCarDataBunch()

In [16]:
let firstBatch = usedCarDataBunch.trainingDataset.first!
let numberFeatures = firstBatch.features.shape[1]
numberFeatures

: 